In [ ]:
# install haystack

!pip install --upgrade pip
!pip install farm-haystack[colab]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 719.5/719.5 kB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 299.9/299.9 kB 33.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 14.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 27.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 48.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.7/10.7 MB 97.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.7/48.7 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 12.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 78.6 MB/s eta 0:00

In [ ]:
# mount gdrive
#small dataset sample stored in my drive
drive.mount("/content/drive", force_remount=True)
from google.colab import drive
drive.mount('/content/drive')
data_path = "/content/drive/My Drive/notclippy"


NameError: ignored

In [ ]:
#set logging
import logging

logging.basicConfig(format="%(levelname)s - %(name)s -  %(message)s", level=logging.WARNING)
logging.getLogger("haystack").setLevel(logging.INFO)


In [ ]:
#initialize documentstore
from haystack.document_stores import InMemoryDocumentStore

document_store = InMemoryDocumentStore(use_bm25=True)

In [ ]:
#initialize document dir

doc_dir = "/content/drive/My Drive/notclippy"



In [ ]:
# parse input tech docs text

file_path = "/content/drive/My Drive/notclippy/input.txt"
with open(file_path, "r") as file:
    content = file.read()

print(content)

In [ ]:
#convert to document objects


import os
from haystack.pipelines.standard_pipelines import TextIndexingPipeline

files_to_index = [doc_dir + "/" + f for f in os.listdir(doc_dir)]
indexing_pipeline = TextIndexingPipeline(document_store)
indexing_pipeline.run_batch(file_paths=files_to_index)



In [ ]:
#initialize retreiver
from haystack.nodes import BM25Retriever

retriever = BM25Retriever(document_store=document_store)

In [ ]:
#intialize reader and pass RoBERTa model
from haystack.nodes import FARMReader

model_name_or_path = "deepset/roberta-base-squad2"
reader = FARMReader(model_name_or_path=model_name_or_path, use_gpu=True)

In [ ]:
#create pipeline
from haystack.pipelines import ExtractiveQAPipeline

pipe = ExtractiveQAPipeline(reader, retriever)

**Ask Question:**

In [ ]:
from pprint import pprint

prediction1 = pipe.run(
    query="how to configure github trust?",
    params={
        "Retriever": {"top_k": 10},
        "Reader": {"top_k": 5}
    }
)

prediction2 = pipe.run(
    query="how to investigate alert?",
    params={
        "Retriever": {"top_k": 10},
        "Reader": {"top_k": 5}
    }
)


prediction3 = pipe.run(
    query="how to monitor high risk employee?",
    params={
        "Retriever": {"top_k": 10},
        "Reader": {"top_k": 5}
    }
)


prediction4 = pipe.run(
    query="how do I identify insider risk?",
    params={
        "Retriever": {"top_k": 10},
        "Reader": {"top_k": 5}
    }
)



**Print Answers**

In [ ]:
pprint(prediction2)

**Simplify answer**



In [ ]:
from haystack.utils import print_answers

print_answers(
    prediction2,
    details="minimum" ## Choose from `minimum`, `medium`, and `all`
)